# Publishing Feedback

### Setup

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

### Adding Feedback to a Run

You can add feedback to an existing Run programmatically as long as you know the run_id. Let's grab a run_id from the LangSmith UI and add it here.

In [2]:
run_id = "483943b3-9efa-435c-a1cd-a97cef169634"

Now, let's add some continuous feedback

In [3]:
from langsmith import Client

client = Client()

client.create_feedback(
  run_id,
  key="sample-continuous",
  score=7.0,
  comment="This is a sample piece of continuous feedback",
)

Feedback(id=UUID('f5808bbd-458a-4f5b-8fa7-90a413e833d8'), created_at=datetime.datetime(2024, 12, 17, 15, 0, 46, 395887, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2024, 12, 17, 15, 0, 46, 395890, tzinfo=datetime.timezone.utc), run_id=UUID('483943b3-9efa-435c-a1cd-a97cef169634'), trace_id=None, key='sample-continuous', score=7.0, value=None, comment='This is a sample piece of continuous feedback', correction=None, feedback_source=FeedbackSourceBase(type='api', metadata={}), session_id=None, comparative_experiment_id=None, feedback_group_id=None, extra=None)

And now, let's add some categorical feedback too!

In [4]:
from langsmith import Client

client = Client()

client.create_feedback(
  run_id,
  key="sample-categorical",
  value="no",
  comment="This is a sample piece of categorical feedback",
)

Feedback(id=UUID('ac61f869-2958-42bf-994b-cc665e0a839f'), created_at=datetime.datetime(2024, 12, 17, 15, 1, 48, 558434, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2024, 12, 17, 15, 1, 48, 558440, tzinfo=datetime.timezone.utc), run_id=UUID('483943b3-9efa-435c-a1cd-a97cef169634'), trace_id=None, key='sample-categorical', score=None, value='no', comment='This is a sample piece of categorical feedback', correction=None, feedback_source=FeedbackSourceBase(type='api', metadata={}), session_id=None, comparative_experiment_id=None, feedback_group_id=None, extra=None)

### Pre-Generating Run IDs for Feedback

![Generate_run_id](../../images/generate_run_id.png)

Using LangChain, we offer the ability to pre-generate and define run IDs, before your code is invoked and the run ID is generated. With this functionality, you're able to access your run ID before initial generation, which can be useful for actions like sending feedback. The example below demonstrates this.

In [5]:
import uuid

pre_defined_run_id = uuid.uuid4()
pre_defined_run_id

UUID('e6c3a498-c0bf-4869-8c15-8e9f7d1c532c')

In [6]:
from langsmith import traceable

@traceable
def foo():
    return "This is a sample Run!"

We are passing in a config with our function call through `langsmith_extra` that contains our pre-defined run_id

In [7]:
foo(langsmith_extra={"run_id": pre_defined_run_id})

'This is a sample Run!'

Now we can directly create feedback on this run!

In [8]:
from langsmith import Client

client = Client()

client.create_feedback(pre_defined_run_id, "user_feedback", score=1)

Feedback(id=UUID('cbcbd7d9-1b5f-4aad-81a9-b0eaa2e8f3cb'), created_at=datetime.datetime(2024, 12, 17, 15, 3, 39, 509317, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2024, 12, 17, 15, 3, 39, 509325, tzinfo=datetime.timezone.utc), run_id=UUID('e6c3a498-c0bf-4869-8c15-8e9f7d1c532c'), trace_id=None, key='user_feedback', score=1, value=None, comment=None, correction=None, feedback_source=FeedbackSourceBase(type='api', metadata={}), session_id=None, comparative_experiment_id=None, feedback_group_id=None, extra=None)

### Pre-signed Feedback URLs

![presigned url](../../images/presigned_url.png)


This can also be helpful for pre-signed feedback URLs. You would want to use these when you can't expose API keys or other secrets to the client, e.g. in a web application. Using a pre-determined run_id LangSmith has an endpoint create_presigned_feedback_token which will create a URL for sending feedback, without the use of secrets required.

In [9]:
pre_signed_url_id = uuid.uuid4()
pre_signed_url_id

UUID('bdfb1ae0-73b5-47c6-8fac-757306b142ba')

In [10]:
pre_signed_url = client.create_presigned_feedback_token(pre_signed_url_id, "user_presigned_feedback")

print(pre_signed_url)

id=UUID('bdfb9108-fcc1-4212-91da-01ec9d08f589') url='https://api.smith.langchain.com/feedback/tokens/bdfb9108-fcc1-4212-91da-01ec9d08f589' expires_at=datetime.datetime(2024, 12, 17, 18, 5, 10, 116182, tzinfo=datetime.timezone.utc)


Here, we can see that even though we haven't created a run yet, we're still able to generate the feedback URL.

Now, let's invoke our chain so the run with that ID is created:

In [11]:
foo(langsmith_extra={"run_id": pre_signed_url_id})

'This is a sample Run!'

Then, once our run is created, we can use the feedback URL to send feedback:

In [12]:
import requests

url_with_score = f"{pre_signed_url.url}?score=1"

response = requests.get(url_with_score)

if response.status_code >= 200 and response.status_code < 300:
    print("Feedback submitted successfully!")
else:
    print("Feedback submission failed!")

Feedback submitted successfully!
